In [9]:
import talib
import websocket
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import json

# Specify the parameters for Trading strategy

In [43]:

aroon_time_period = 14
amount = 1000
core_trade_amount = amount * 0.80
trade_amount = amount * 0.20
money_end = amount 
portfolio = 0
investment, closes, highs, lows = [], [], [], []

# For trading
core_to_trade = True
core_quantity = 0


# Paper Trading Simulation Function

In [13]:
def buy(allocated_money, price):
    global money_end, portfolio, investment

    quantity = allocated_money / price
    money_end -= quantity * price
    portfolio += quantity

    if investment == []:
        investment.append(allocated_money)
    else:
        investment.append(allocated_money) 
        investment[-1] += investment[-2]
    return {'quantity':quantity, 'investment':investment, 'money_end': money_end, 'portfolio':portfolio}

def sell(allocated_money, price):
    global money_end, portfolio, investment

    quantity = allocated_money / price
    money_end += quantity * price
    portfolio -= quantity
    investment.append(-allocated_money)
    investment[-1] += investment[-2]
    return {'quantity':quantity, 'investment':investment, 'money_end': money_end, 'portfolio':portfolio}

In [14]:
buy(100, 6) # buy product of 100 with unit price of 6

{'quantity': 16.666666666666668,
 'investment': [100],
 'money_end': 900.0,
 'portfolio': 16.666666666666668}

In [15]:
sell(200,7) # buy product of 200 with unit price of 7

{'quantity': 28.571428571428573,
 'investment': [100, -100],
 'money_end': 1100.0,
 'portfolio': -11.904761904761905}

# Extract Real Time Crypto Data Using Web-Socket, making Candle Stick, Trading Strategy

In [46]:
cc = 'btcusd'
interval = '1m'
socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

def on_message(ws, message):
    global closes, highs, lows, core_to_trade, core_quantity, money_end, portfolio, investment

    message_json = json.loads(message)
    cs = message_json['k'] # cs: candle stick
    print({'candle_is_closed': cs['x'], 'close':cs['c'], 'high':cs['h'], 'low':cs['l']})

    # check the candle close

    if cs['x']:
        highs.append(cs['h'])
        lows.append(cs['l'])
        closes.append(cs['c'])
        last_price = closes[-1]
        print('The Highs Lows and Closes become :-', {'highs': highs, 'lows': lows, 'closes': closes, 'last_price': last_price})


    
    

def on_close(ws):
    print('close')

ws = websocket.WebSocketApp(socket, on_message=on_message, on_close=on_close)

In [47]:
ws.run_forever()

{'candle_is_closed': False, 'close': '19197.20000000', 'high': '19198.42000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19190.40000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19188.55000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19176.52000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19171.52000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19181.88000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19178.46000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19185.25000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19190.99000000', 'high': '19218.18000000', 'low': '19156.41000000'}
{'candle_is_closed': False, 'close': '19176.79

False

In [49]:
print('The Highs Lows and Closes are:- ', {'highs': highs, 'lows': lows, 'closes': closes, })

The Highs Lows and Closes are:-  {'highs': ['19149.20000000', '19218.18000000'], 'lows': ['19091.02000000', '19155.16000000'], 'closes': ['19136.57000000', '19165.51000000']}


# Message will give:
{"t":1655541660000,"T":1655541719999,"s":"BTCUSDT","i":"1m","f":1413808576,"L":1413809419,"o":"19083.84000000","c":"19132.82000000","h":"19134.05000000","l":"19083.20000000","v":"83.27792000","n":844,"x":false,"q":"1591651.11390620","V":"44.36906000","Q":"847883.69327980","B":"0"}}

Here:
"x": false -> means the same candle, so after 1 minute it will be true to indicate a new candle. We will work with one candle.

In [35]:
message = {"t":1655541660000,"T":1655541719999,"s":"BTCUSDT","i":"1m","f":1413808576,"L":1413809419,"o":"19083.84000000","c":"19132.82000000","h":"19134.05000000","l":"19083.20000000","v":"83.27792000","n":844,"x":False,"q":"1591651.11390620","V":"44.36906000","Q":"847883.69327980","B":"0"}

# after json.loads function 

message_json = {'e': 'kline', 'E': 1655543350450, 's': 'BTCUSDT', 'k': {'t': 1655543340000, 'T': 1655543399999, 's': 'BTCUSDT', 'i': '1m', 'f': 1413867906, 'L': 1413868113, 'o': '18989.99000000', 'c': '19001.00000000', 'h': '19003.18000000', 'l': '18989.98000000', 'v': '6.26305000', 'n': 208, 'x': False, 'q': '118990.39466130', 'V': '3.85642000', 'Q': '73266.47400480', 'B': '0'}}

False